In [17]:
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.python.keras.utils import np_utils
from keras.models import Sequential
from keras.utils import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [2]:
path = r"C:\\Users\\angel\\Desktop\\font_original" #원본 이미지 경로 
copy_path = r"C:\\Users\\angel\\Desktop\\font_classifier" #복사 및 분류된 이미지 경로


#font_dataset 안에 모든 글꼴 이미지들이 다 저장되어 있음 (따로 하위폴더 없음)

### 데이터셋 준비

In [ ]:
import splitfolders

splitfolders.ratio(path, output=copy_path, seed = 43, ratio = (.5,.2,.3))

In [3]:
print(len(os.listdir(copy_path + '\\val')))
print(len(os.listdir(copy_path + '\\train')))
print(len(os.listdir(copy_path + '\\test')))

9
9
9


### 데이터셋 정형화

In [4]:
trainDataGen = ImageDataGenerator(rescale=1./255)

In [5]:
currentPath = copy_path + '\\train'

In [6]:
trainGenSet = trainDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(224,224),
    class_mode='categorical'
)

Found 50274 images belonging to 9 classes.


-------------

In [7]:
valDataGen = ImageDataGenerator(rescale=1./255)

In [8]:
currentPath = copy_path + '\\val'

In [9]:
valGenSet = valDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(224,224),
    class_mode='categorical'
)

Found 20106 images belonging to 9 classes.


---------------------

In [10]:
testDataGen = ImageDataGenerator(rescale=1./255,
                                 rotation_range = 30,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest'
                                 )

In [11]:
currentPath = copy_path + '\\test'

In [12]:
testGenSet = testDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(224,224),
    class_mode='categorical'
)

Found 30168 images belonging to 9 classes.


--------

### 모델 구성

In [27]:
import tensorflow as tf
from tensorflow.keras import layers

def VGG16_model():
    tf.random.set_seed(2)
    model = tf.keras.models.Sequential([
        # Conv 1
        layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu',
                     input_shape=(224,224,1), name='conv1_1'),
        layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu', name='conv1_2'),
        layers.MaxPool2D((2,2), padding='same', name='conv1_MaxPool'),
        # Conv 2
        layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', name='conv2_1'),
        layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', name='conv2_2'),
        layers.MaxPool2D((2,2), padding='same', name='conv2_MaxPool'),
        # Conv3
        layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', name='conv3_1'),
        layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', name='conv3_2'),
        layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', name='conv3_3'),
        layers.MaxPool2D((2,2), padding='same', name='conv3_MaxPool'),
        # Conv4
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv4_1'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv4_2'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv4_3'),
        layers.MaxPool2D((2,2), padding='same', name='conv4_MaxPool'),
        # Conv5
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv5_1'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv5_2'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv5_3'),
        layers.MaxPool2D((2,2), padding='same', name='conv5_MaxPool'),
        # Flatten
        layers.Flatten(),
        # Dropout
        layers.Dropout(0.5),
        # FC1
        layers.Dense(512, activation='relu'),
        # output
        layers.Dense(9, activation='softmax'),
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

    return model

### 모델 훈련

In [43]:
# Compile the model

model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
Model= VGG16_model()

results = Model.fit(
    trainGenSet,
    steps_per_epoch=40,
    epochs=100,
    validation_data=valGenSet,
    validation_steps=20
)

Epoch 1/100


UnimplementedError: Graph execution error:

Detected at node 'sequential_4/conv1_1/Relu' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "C:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_foreve

### 모델 accuracy, loss 그래프로 표현

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 모델 테스트 및 정확도 출력 

In [ ]:
scores = model.evaluate_generator(testGenSet)
print(scores)